In [1]:
# Model
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig
from langchain_huggingface import HuggingFacePipeline
import torch
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.prompts import PromptTemplate
# Vector stores
import fitz  # PyMuPDF
import pdfplumber
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.document_loaders import PyMuPDFLoader, PDFPlumberLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter, KonlpyTextSplitter
from langchain_community.retrievers import BM25Retriever, KNNRetriever
from langchain.retrievers import EnsembleRetriever
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.document_loaders import PyMuPDFLoader, PDFPlumberLoader ,UnstructuredPDFLoader
from langchain_community.retrievers import BM25Retriever, KNNRetriever
from langchain.retrievers import EnsembleRetriever
from langchain_teddynote.retrievers import KiwiBM25Retriever
from langchain.docstore.document import Document
from concurrent.futures import ThreadPoolExecutor
import pandas as pd
import unicodedata
import time
import re
from konlpy.tag import Okt
from pdf2image import convert_from_path
import pytesseract
from konlpy.tag import Kkma
# etc
import os
import pandas as pd
from tqdm import tqdm
import unicodedata
import logging

import json


c:\ProgramData\anaconda3\envs\llm_titan\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2
# intfloat/multilingual-e5-small
# jhgan/ko-sroberta-multitask


In [3]:
from langchain_unstructured import UnstructuredLoader

def get_embedding():
    device = 'cuda' if torch.cuda.is_available() and 'mps' in torch.cuda.get_device_capability() else 'cpu'  # GPU 사용 가능 여부 및 MPS 지원 여부 확인
    embeddings = HuggingFaceEmbeddings(
        model_name='jhgan/ko-sroberta-multitask',
        model_kwargs={'device': device},
        encode_kwargs={'normalize_embeddings': True})
    return embeddings
def normalize_string(s):
    try:
        normalized = unicodedata.normalize('NFC', s)
        return normalized.encode('utf-8', errors='replace').decode('utf-8')
    except Exception:
        return s
def clean_text(text):
    text = text.replace("�", "").replace("", "")  # 잘못된 인코딩 문자 제거
    text = ' '.join(text.split())  # 여러 공백을 하나로 줄임
    return text


In [4]:
def ocr_extract_text_from_pdf(pdf_path):
    documents = []
    try:
        # PDF를 이미지로 변환
        images = convert_from_path(pdf_path, dpi=500)
        
        # 각 페이지에서 OCR로 텍스트 추출
        for i, image in enumerate(images):
            text = pytesseract.image_to_string(image, lang='kor+eng')
            text = clean_text(normalize_string(text))
            # 각 페이지마다 별도의 Document 생성
            documents.append(Document(page_content=text, metadata={"source": pdf_path, "page": i + 1}))
    
    except Exception as e:
        print(f"Error extracting text from {pdf_path}: {e}")
    
    return documents

def load_pdf_file(pdf_file):
    try:
        # OCR을 이용한 텍스트 추출
        documents = ocr_extract_text_from_pdf(pdf_file)
        return documents
    except Exception as e:
        print(f"Skipping corrupted file {pdf_file}: {e}")
        return []

def split_documents_with_konlpy(documents):
    adjusted_documents = []
    konlpy_splitter = KonlpyTextSplitter(separator='\n\n')

    for doc in documents:
        # KonlpyTextSplitter로 문장 단위 분리
        sentences = konlpy_splitter.split_text(doc.page_content)
        for sentence in sentences:
            adjusted_documents.append(Document(page_content=sentence, metadata=doc.metadata))
    
    return adjusted_documents

def make_db(df, chunk_name="train", chunk_size=800, chunk_overlap=60):
    documents = []
    chunk_file_path = f"{chunk_name}_chunks.json"

    if os.path.exists(chunk_file_path):
        print(f"Loading chunks from {chunk_file_path}...")
        documents = load_chunks(chunk_file_path)
    else:
        pdf_files = df['Source_path'].unique()
        with ThreadPoolExecutor() as executor:
            results = list(tqdm(executor.map(load_pdf_file, pdf_files), total=len(pdf_files), desc="Processing PDFs"))
        
        for result in results:
            documents.extend(result)
        
        # KonlpyTextSplitter로 문장 단위로 먼저 분할
        documents = split_documents_with_konlpy(documents)

        # 청크 데이터를 저장
        save_chunks(documents, chunk_file_path)
    
    # make_db에서 RecursiveCharacterTextSplitter로 청크를 분할
    recursive_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separators=["\n\n", "."]
    )
    documents = recursive_splitter.split_documents(documents)
    
    faiss = FAISS.from_documents(documents, embedding=get_embedding())
    bm = KiwiBM25Retriever.from_documents(documents)
    
    return faiss, bm

def save_chunks(documents, path):
    chunk_data = []
    for doc in documents:
        chunk_data.append({
            "page_content": doc.page_content,
            "metadata": doc.metadata
        })
    
    with open(path, "w", encoding="utf-8") as f:
        json.dump(chunk_data, f, ensure_ascii=False, indent=4)
    
    print(f"Chunks saved to {path}")

def load_chunks(path):
    with open(path, "r", encoding="utf-8") as f:
        chunk_data = json.load(f)
    
    documents = []
    for chunk in chunk_data:
        documents.append(Document(page_content=chunk["page_content"], metadata=chunk["metadata"]))
    
    print(f"Chunks loaded from {path}")
    return documents

In [5]:
def fewshot_db(df):
    df = df.drop('SAMPLE_ID', axis=1)
    df = df.drop('Source_path', axis=1)
    df = df.to_dict(orient='records')
    print("Loaded Fewshot Set:", len(df))
    to_vectorize = ["\n\n".join(normalize_string(value) for value in example.values()) for example in df]
    faiss = FAISS.from_texts(to_vectorize, embedding=get_embedding())
    # bm = BM25Retriever.from_texts(to_vectorize)
    # knn = KNNRetriever.from_texts(to_vectorize, embeddings=get_embedding())
    return faiss

In [6]:
train_df = pd.read_csv('train.csv', encoding='utf-8')
test_df = pd.read_csv('test.csv', encoding='utf-8')

pypdf 이건 x

표가 많기 때문에

PyMuPDFLoader 나 PDFPlumberLoader를 쓰고

표를 이미지로 불러오기도 하는데 이런경우 추가적인 전처리가 필요함

In [7]:
def format_docs(docs):
    """검색된 문서들을 하나의 문자열로 포맷팅"""
    context = ""
    for i, doc in enumerate(docs):
        #context += f"Document {i+1}\n"
        doc.page_content = doc.page_content.replace("{", "(")
        doc.page_content = doc.page_content.replace("}", ")")
        
        context += doc.page_content
        context += '\n\n'
    return context.strip()

In [8]:
def setup_llm_pipeline(model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"):
    # 토크나이저 로드 및 설정
        # 양자화 설정 적용
    bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, 
    bnb_4bit_use_double_quant=True, 
    bnb_4bit_quant_type="nf4", 
    bnb_4bit_compute_dtype=torch.bfloat16
    )
    model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config,low_cpu_mem_usage=True)
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    text_generation_pipeline = pipeline(
        model=model,
        tokenizer=tokenizer,
        task="text-generation",
        return_full_text=False,
        temperature = 0.5,
        max_new_tokens=1024,
        eos_token_id = terminators,
        pad_token_id = tokenizer.eos_token_id
    )

    llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

    return llm
llm = setup_llm_pipeline()

Loading checkpoint shards: 100%|██████████| 4/4 [01:03<00:00, 15.78s/it]


In [9]:
# def template_extractor(df):
#     full_template = "<|begin_of_text|>"
#     full_template += """<|start_header_id|>system<|end_header_id|>아래 문장들에서 나타나는 답변 문체들을 찾아주세요.<|eot_id|>"""
#     for i, answer in enumerate(df['Answer']):
#         full_template += f"{answer}<|eot_id|>"
#     #buff_str += f"<|start_header_id|>assistant<|end_header_id>"
#     template = PromptTemplate(full_template)
#     chain =(
#         template
#         | llm
#         | StrOutputParser()
#     )
#     return chain

# chain = template_extractor(train_df).invoke()
# print(chain)

In [10]:
def extract_answer(response):
    # AI: 로 시작하는 줄을 찾아 그 이후의 텍스트만 추출
    lines = response.split('\n')
    for line in lines:
        line = line.replace('**', '')
        if line.startswith('Answer:'):
            return line.replace('Answer:', '').strip()
        if line.startswith('assistant:'):
            return line.replace('assistant:', '').strip()
    return response.strip()  # AI: 를 찾지 못한 경우 전체 응답을 정리해서 반환

def fewshot_ex(fewshot_retriever, train_retriever, q, verbose=False):
    query = normalize_string(q)
    fewshot_results = fewshot_retriever.invoke(normalize_string(query)) #Document(page_content='중소벤처기업부_창업사업화지원\n\n창업사업화지원의 사업목적은 무엇인가?\n\n창업사업화지원의 사업목적은 창업기업의 성장단계별, 초격차 분야별, 글로벌화 지원체계를 구축‧운영하여 혁신 기술창업을 활성화하고 창업기업 성장 및 생존율 제고하는 것이다.')
    fewshot_str = ""
    i = 1
    for result in fewshot_results:
    
        result = result.page_content.split('\n\n')
        buff_str = f"<|start_header_id|>system<|end_header_id|>Example{i}\n<|eot_id|>\n\n"
        i+=1
        buff_str += "<|start_header_id|>user<|end_header_id|>"
        question = result[1]
        buff_str += f"Question: {question}\n\n"
        if train_retriever is not None:
            buff_str += f"Context:"
            docs = train_retriever.invoke(normalize_string(question))
            if verbose:
                print("Fewshot Q |",len(docs),"|",question)
            buff_str += format_docs(docs)
            buff_str += "<|eot_id|>\n\n"
        else: 
            buff_str += "<|eot_id|>\n\n"
            if verbose:
                print("Fewshot Q |",question)
        buff_str += f"<|start_header_id|>assistant<|end_header_id>{result[2]}<|eot_id|>"
        fewshot_str += buff_str    
        
    return fewshot_str

def run (train,test,fewshot,dataset,llm,verbose=False):
    results = []
    for i, row in (dataset.iterrows()):

        full_template = "<|begin_of_text|>"
        full_template += """<|start_header_id|>system<|end_header_id|>
You are the financial expert who helps me with my financial information Q&As.
You earn 10 points when you answer me and follow the rules and lose 7 points when you don't.
Please use contexts to answer the question.
Please your answers should be concise.
Please answers must be written in Korean.
Please answer in 1-3 sentences.
Answer like the question-answer examples given.
<|eot_id|>
"""
        question = row['Question']        
        if verbose:
            print(f"====={i}/{len(dataset)}{'='*255}")
            print("Question: ", question)
        
        fewshot_str = fewshot_ex(fewshot, train, question,verbose)
        full_template += fewshot_str
        full_template += """ """
        contexts = test.invoke(normalize_string(question))
        if verbose:
            print("Context Number|",len(contexts),end="|")
        contexts = format_docs(contexts)
        full_template += """<|start_header_id|>user<|end_header_id|>Question: {input}\n\n"""
        full_template += f"""Context: {contexts}<|eot_id|>\n\n"""
        full_template += """<|start_header_id|>assistant<|end_header_id>"""
        
        prompt = PromptTemplate(template=full_template, input_variables=["input"])
        qa_chain = (
        {
            "input": RunnablePassthrough(),
        }
        | prompt
        | llm
        | StrOutputParser()
        )

        answer = qa_chain.invoke(input=question)
        answer = extract_answer(answer)
        results.append({
            "Question": question,
            "Answer": answer,
            "Source": row['Source']
        })
        if verbose:
            print("=====Answer=====\n", results[-1]['Answer'])
            try:
                print("=====REAL Answer=====\n",row['Answer'])
            except:
                pass
    return results

In [11]:
from collections import Counter
def calculate_f1_score(true_sentence, predicted_sentence, sum_mode=True):

    #공백 제거
    true_sentence = ''.join(true_sentence.split())
    predicted_sentence = ''.join(predicted_sentence.split())
    
    true_counter = Counter(true_sentence)
    predicted_counter = Counter(predicted_sentence)

    #문자가 등장한 개수도 고려
    if sum_mode:
        true_positive = sum((true_counter & predicted_counter).values())
        predicted_positive = sum(predicted_counter.values())
        actual_positive = sum(true_counter.values())

    #문자 자체가 있는 것에 focus를 맞춤
    else:
        true_positive = len((true_counter & predicted_counter).values())
        predicted_positive = len(predicted_counter.values())
        actual_positive = len(true_counter.values())

    #f1 score 계산
    precision = true_positive / predicted_positive if predicted_positive > 0 else 0
    recall = true_positive / actual_positive if actual_positive > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    return precision, recall, f1_score

def calculate_average_f1_score(true_sentences, predicted_sentences):
    
    total_precision = 0
    total_recall = 0
    total_f1_score = 0
    
    for true_sentence, predicted_sentence in zip(true_sentences, predicted_sentences):
        precision, recall, f1_score = calculate_f1_score(true_sentence, predicted_sentence)
        total_precision += precision
        total_recall += recall
        total_f1_score += f1_score
    
    avg_precision = total_precision / len(true_sentences)
    avg_recall = total_recall / len(true_sentences)
    avg_f1_score = total_f1_score / len(true_sentences)
    
    return {
        'average_precision': avg_precision,
        'average_recall': avg_recall,
        'average_f1_score': avg_f1_score
    }

In [12]:
from sklearn.model_selection import KFold
import copy

weight = [0.5,0.5]
train_faiss_db, train_bm_retrievier = make_db(train_df,'./train') 

train_k = 1
train_bm_retrievier.k = train_k
#knn_retriever.k = train_k
train_faiss_retriever = train_faiss_db.as_retriever(search_type="mmr",search_kwargs={'k':train_k} )
train_ensemble_retriever = EnsembleRetriever(
    retrievers=[train_bm_retrievier, train_faiss_retriever], weights=weight
)

test_bm_retrievier = copy.deepcopy(train_bm_retrievier)
test_k = 2
test_bm_retrievier.k = test_k
#test_knn_retriever.k = test_k
test_faiss_retriever = train_faiss_db.as_retriever(search_type="mmr",search_kwargs={'k':test_k} )
test_ensemble_retriever = EnsembleRetriever(
    retrievers=[test_bm_retrievier, test_faiss_retriever], weights=weight
)

fewshot_k = 3

k_folds = 4
fold_results = []
kf = KFold(n_splits=k_folds, shuffle=True, random_state=52)
for fold, (train_index, val_index) in enumerate(kf.split(train_df)):
    fold_result = []
    train_set = train_df.iloc[train_index]
    val_set = train_df.iloc[val_index]
    
    
    fewshot_faiss_db = fewshot_db(train_set)
    fewshot_faiss_retriever = fewshot_faiss_db.as_retriever(search_kwargs={'k':fewshot_k} )

    pred = run(train_ensemble_retriever, test_ensemble_retriever, fewshot_faiss_retriever, val_set, llm, verbose=True)
    result = pd.DataFrame()
    result['pred'] = [result['Answer'] for result in pred]
    val_set.index = range(len(val_set))
    result['gt'] = val_set['Answer']
        
    result = calculate_average_f1_score(result['gt'], result['pred'])
    print(result)
    fold_results.append(result)
    break

No chunks. Make chunks ./train_chunks.json...


Processing PDFs:   0%|          | 0/16 [00:00<?, ?it/s]

Error extracting text from ./train_source/중소벤처기업부_창업사업화지원.pdf: tesseract is not installed or it's not in your PATH. See README file for more information.
Error extracting text from ./train_source/국토교통부_소규모주택정비사업.pdf: tesseract is not installed or it's not in your PATH. See README file for more information.
Error extracting text from ./train_source/고용노동부_내일배움카드(일반).pdf: tesseract is not installed or it's not in your PATH. See README file for more information.
Error extracting text from ./train_source/고용노동부_청년일자리창출지원.pdf: tesseract is not installed or it's not in your PATH. See README file for more information.
Error extracting text from ./train_source/보건복지부_생계급여.pdf: tesseract is not installed or it's not in your PATH. See README file for more information.
Error extracting text from ./train_source/국토교통부_민간임대(융자).pdf: tesseract is not installed or it's not in your PATH. See README file for more information.
Error extracting text from ./train_source/국토교통부_전세임대(융자).pdf: tesseract is not in

py pdf 0.663

ptmu 0.6740735793987513

PDFPlumberLoader 0.6516317411146649 0.655995454987313(0.4), 0.6578699067809993(0.2), 0.6578699067809993(0.1)

PDFPlumberLoader NEW:  0.6508706361694363, 0.6545265117992998 (0.4)

NEWNEW : 0.6690379419494678(0.5)recursive,chunk1000,overlap200 -> 0.59

konlp : 0.639172573877312 chunk 400 

0.6502491672498713 chunk 2000 

0.6679938815194875 chunk 1000

0.6716520173133778 chunk 500 0.5819408919	

====

Unstructed 

0.64 (-)

0.6612316276810916 (0.5)

0.6578298248291823 (0.3)


====

Unstructed (New)

In [ ]:
# from save_module import save


# weight = [0.5,0.5]
# train_faiss_db, train_bm_retrievier = make_db(train_df) 
# test_faiss_db, test_bm_retrievier = make_db(test_df)
# fewshot_faiss_db = fewshot_db(train_df)

# train_k = 1
# train_bm_retrievier.k = train_k
# #knn_retriever.k = train_k
# faiss_retriever = train_faiss_db.as_retriever(search_type="mmr",search_kwargs={'k':train_k} )
# train_ensemble_retriever = EnsembleRetriever(
#     retrievers=[train_bm_retrievier, faiss_retriever], weights=weight
# )

# test_k = 2
# test_bm_retrievier.k = test_k
# #test_knn_retriever.k = test_k
# test_faiss_retriever = test_faiss_db.as_retriever(search_type="mmr",search_kwargs={'k':test_k} )
# test_ensemble_retriever = EnsembleRetriever(
#     retrievers=[test_bm_retrievier, test_faiss_retriever], weights=weight
# )

# fewshot_k = 3
# # fewshot_bm_retrievier.k = fewshot_k
# #fewshot_knn_retriever.k = fewshot_k
# fewshot_faiss_retriever = fewshot_faiss_db.as_retriever(search_type="mmr",search_kwargs={'k':fewshot_k} )
# # fewshot_ensemble_retriever = EnsembleRetriever(
# #     retrievers=[fewshot_bm_retrievier, fewshot_faiss_retriever], weights=weight
# # )

# results = run(train_ensemble_retriever, test_ensemble_retriever, fewshot_faiss_retriever, test_df, llm, verbose=True)
# save(results)